## 使用paramiko实现ssh和sftp

In [1]:
import paramiko

In [2]:
def to_str(bytes_or_str):
    """
    把byte类型转换为str
    :param bytes_or_str:
    :return:
    """
    if isinstance(bytes_or_str, bytes):
        value = bytes_or_str.decode('utf-8')
    else:
        value = bytes_or_str
    return value

In [3]:
class SSHConnection(object):
 
    def __init__(self, host_dict):
        self.host = host_dict['host']
        self.port = host_dict['port']
        self.username = host_dict['username']
        self.pwd = host_dict['pwd']
        self.__k = None
 
    def connect(self):
        transport = paramiko.Transport((self.host,self.port))
        transport.connect(username=self.username,password=self.pwd)
        self.__transport = transport
 
    def close(self):
        self.__transport.close()
 
    def run_cmd(self, command):
        """
         执行shell命令,返回字典
         return {'color': 'red','res':error}或
         return {'color': 'green', 'res':res}
        :param command:
        :return:
        """
        ssh = paramiko.SSHClient()
        ssh._transport = self.__transport
        # 执行命令
        stdin, stdout, stderr = ssh.exec_command(command)
        # 获取命令结果
        res = to_str(stdout.read())
        # 获取错误信息
        error = to_str(stderr.read())
        # 如果有错误信息，返回error
        # 否则返回res
        if error.strip():
            return {'color':'red','res':error}
        else:
            return {'color': 'green', 'res':res}
 
    def upload(self,local_path, target_path):
        # 连接，上传
        sftp = paramiko.SFTPClient.from_transport(self.__transport)
        # 将location.py 上传至服务器 /tmp/test.py
        sftp.put(local_path, target_path, confirm=True)
        # print(os.stat(local_path).st_mode)
        # 增加权限
        # sftp.chmod(target_path, os.stat(local_path).st_mode)
        sftp.chmod(target_path, 0o755)  # 注意这里的权限是八进制的，八进制需要使用0o作为前缀
 
    def download(self,target_path, local_path):
        # 连接，下载
        sftp = paramiko.SFTPClient.from_transport(self.__transport)
        # 将location.py 下载至服务器 /tmp/test.py
        sftp.get(target_path, local_path)
 
    # 销毁
    def __del__(self):
        self.close()
 


## 创建连接

In [4]:
s = SSHConnection(dict(host='192.168.0.2',port=22,username='test',pwd='test'))
s.connect()

## 执行命令

In [5]:
print(s.run_cmd('ls -l /')['res'])

total 96
drwxrwxrwx    5 root root  4096 Jun 17  2021 app
lrwxrwxrwx    1 root root     7 Jan  4  2019 bin -> usr/bin
dr-xr-xr-x.   6 root root  4096 Mar 24  2019 boot
drwxr-xr-x    7 root root  4096 Jan  4  2019 CloudResetPwdUpdateAgent
drwxr-xr-x    7 root root  4096 Jan  4  2019 CloudrResetPwdAgent
drwxr-xr-x   19 root root  3020 Feb 23  2019 dev
drwxrwxrwx    4 root root  4096 Jun 16  2021 download
drwxr-xr-x.  92 root root 12288 Dec 31 13:50 etc
drwxr-xr-x.   6 root root  4096 Sep 30 13:48 home
lrwxrwxrwx    1 root root     7 Jan  4  2019 lib -> usr/lib
lrwxrwxrwx    1 root root     9 Jan  4  2019 lib64 -> usr/lib64
drwx------.   2 root root 16384 Jan  4  2019 lost+found
drwxr-xr-x.   2 root root  4096 Apr 11  2018 media
drwxr-xr-x.   2 root root  4096 Apr 11  2018 mnt
drwxr-xr-x.   9 root root  4096 Sep 30 11:03 opt
dr-xr-xr-x  138 root root     0 Feb 23  2019 proc
dr-xr-x---.  11 root root  4096 Nov 26 18:52 root
drwxr-xr-x   34 root root  1000 Dec  1 08:50 run
lrwxrwxrwx    1 r

## 上传文件

In [6]:
s.upload('test.txt','/tmp/test.txt')
print(s.run_cmd('ls -l /tmp/test.txt')['res'])

-rwxr-xr-x 1 leo leo 12 Dec 31 13:56 /tmp/test.txt



## 下载文件

In [7]:
s.download('/tmp/test.txt','test1.txt')